In [21]:
import mpbn
import pandas as pd
import numpy as np

# 1000 models

In [3]:
modelname = []
modelname.extend([f"models/bn{n}.bnet" for n in range(1000)])

solutions = list()
for i,n in enumerate(modelname):
    solutions.append(mpbn.MPBooleanNetwork(f"{n}"))

`clauses_per_model` : list of models, each model being a dictionary
* keys = **model's nodes** 
* value = a set (-> **the clauses constituting the node's function in the model**)

In [4]:
# For each model, I store the clauses per node
clauses_per_model = list() #Each element of this list will correspond to a model, it's a dict linking nodes and their clauses.
for i, model in enumerate(solutions): #We go through the 1000 models.
    clauses_of_a_node = dict() #The dict that will link a node to its clauses.
    for node in model.keys(): #We go through the nodes of the model.
        #if not isinstance(model[node],frozenset):
        if model[node]==True or model[node]==False :
            clauses_of_a_node[node] = model[node]
        else:
            clauses_of_a_node[node] = mpbn.minibn.struct_of_dnf(model.ba, model[node]) #We store the clauses ruling this node.
    clauses_per_model.append(clauses_of_a_node) #Now that all nodes are linked with their clauses, we save this information as a new element in clauses_per_model.

`dico` : dictionnary
* keys = **model's nodes** 
* value = dictionary  
     + keys = **its inhibitors and activators**
     + value = **number of models having this inhibitor/activator in the node's function**

`constantes` : dictionnary
* keys = **model's nodes with at least one constant function in a model (constant function : 1 or 0)**
* value = dictionnary
     + key = **value of the constant function (TRUE for 1, FALSE for 0)**
     + value = **number of models having 1 or 0 as function for the node**

In [7]:
dico = dict()
constantes = dict()
for model in clauses_per_model:
    for node, influenceurs_set in model.items():
        temp = set()
        if node not in dico:
            dico[node] = dict()
        #print(influenceurs_set)
        #print(type(influenceurs_set))
        #print(isinstance(influenceurs_set, bool))
        if influenceurs_set != True and influenceurs_set != False:
            for conjonctions in influenceurs_set:
                for clause in conjonctions:
                    if clause[0] not in temp:
                        temp.add(clause[0])
                        if clause[0] not in dico[node]:
                            dico[node][clause[0]] = dict()
                            dico[node][clause[0]][clause[1]] = 1
                        elif clause[1] not in dico[node][clause[0]]:
                            dico[node][clause[0]][clause[1]] = 1
                        else:
                            dico[node][clause[0]][clause[1]] += 1
        else:
            if node not in constantes:
                constantes[node] = dict()
            if influenceurs_set not in constantes[node]:
                constantes[node][influenceurs_set] = 1
            else:
                constantes[node][influenceurs_set] += 1

## Pour mettre en évidence d'éventuels clusters parmi les modèles :

Utilisation de 2 méthodes de clustering :
* MDS (MultiDimensional Scaling)
* clustering hiérarchique ascendante ("agglomerative")

Afin de pouvoir utiliser ces méthodes de clustering, il a fallu déterminer un calcul de distance entre modèles. Cette distance "inter-BN" est la suivante :  
**Étant donné $m_1$ et $m_2$ deux réseaux booléens de dimension $n$,  
la distance "inter-BN" $d(m_1,m_2) = \sum_{i=1}^{n} \mathbb{1}(m_{1}[i] \neq m_{2}[i])$**

Selon la graine aléatoire utilisée pour le MDS, j'obtiens 3 ou 4 clusters. Parmi eux, 2 clusters sont toujours strictement identiques. Pour les suivants, deux cas de figures en fonction qu'il y ait 3 ou 4 clusters au total :

* **Dans le cas de la division en 4 clusters,** la division entre les 2 groupes restants est instable selon la graine aléatoire utilisée pour le calcul. Par exemple:
   - **4+1 :** $[0, 93, 675, 966]$ et $[1]$
   - **3+2 :** $[93, 675, 966]$ et $[0, 1]$
* **Dans le cas de la division en 3 clusters,** les clusters restent identiques quelque soit les graines aléatoires testées. Le cluster n°3 étant :
   - $[0, 1, 93, 675, 966]$


Le clustering hiérarchique en 3 clusters aboutit en exactement aux mêmes 3 clusters que le MDS en 3 clusters. Le clustering hiérarchique en 4 clusters aboutit à la division suivante :
* $[0, 1, 93, 966]$ et $[675]$

Étant donné la constance de la division en 3 clusters, ce clustering apparaît comme le plus pertinent.

In [15]:
# packages pour calculer MDS et AgglomerativeClustering + scatterplot et dendrogram
import sklearn as sk
from sklearn import manifold
from sklearn import cluster
import sklearn.datasets as dt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram

In [16]:
%matplotlib qt

In [19]:
df_models = pd.DataFrame(clauses_per_model)

### Création de la matrice de distance

In [22]:
dissimilarity_matrix = np.zeros((1000,1000))

for i in range(0,1000):
    for j in range(i+1,1000):
        distance = 0
        for node in dico.keys():
            if df_models.loc[i,node] != df_models.loc[j,node]:
                distance += 1
        dissimilarity_matrix[i,j] = distance
        dissimilarity_matrix[j,i] = distance

Si besoin :

In [ ]:
# sauvegarde de la matrice de similarité :
import pickle
with open('data/analysis/dissimilarity_matrix_1000.binary', 'wb') as f:
    pickle.dump(dissimilarity_matrix, f)

In [ ]:
# récupération de la matrice de similarité :
import pickle
with open('dissimilarity_matrix_1000.binary', 'rb') as f:
    dissimilarity_matrix = pickle.load(f)

### MDS 2D

**Grande majorité des graines aléatoires : 3 clusters.**  
Exemple ici avec la graine 10 : 

In [ ]:
mds = manifold.MDS(
    dissimilarity="precomputed", 
    random_state=10 #3 pour 250
)
npos = mds.fit_transform(dissimilarity_matrix)

f = plt.figure(figsize=(10, 10))
ax = sns.scatterplot(x=npos[:, 0], y=npos[:, 1])

**Les graines aboutissant à 4 clusters divisent constamment le même cluster, mais de façon instable (ce ne sont pas constamment les mêmes modèles qui sont séparés).**  
Exemple ici avec la graine 12 :

In [ ]:
mds_4c = manifold.MDS(
    dissimilarity="precomputed", 
    random_state=12  #,n_jobs=-1 : pour calculer en utilisant tous les cœurs.
) 
npos_4c = mds_4c.fit_transform(dissimilarity_matrix)

f = plt.figure(figsize=(10, 10))
ax = sns.scatterplot(x=npos_4c[:, 0], y=npos_4c[:, 1])

### MDS 3D
ne permet pas d'affiner le clustering par rapport au 2D

In [ ]:
mds_3D = manifold.MDS(
    n_components=3,
    dissimilarity="precomputed"
)
npos_3D = mds_3D.fit_transform(dissimilarity_matrix)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
xs = npos_3D[:, 0]
ys = npos_3D[:, 1]
zs = npos_3D[:, 2]
ax.scatter(xs, ys, zs)

### Analyse du MDS 2D, 3 clusters :

Position des différents clusters dans le graphe 2D:
* $y < 2$
* $x < 0, y > 15$
* $x > 5$

#### 1) Plot : mise en évidence des clusters par des couleurs différentes.

In [ ]:
# Je crée un vecteur de booléen pour savoir si le modèle i est dans le cluster0 ou non.
cluster0_x = npos[:,0] < 0
cluster0_y = npos[:,1] > 15
cluster0 = cluster0_x & cluster0_y

cluster0_indices = list()
for i in range(0,npos.shape[0]):
    if cluster0[i]:
        cluster0_indices.append(i)
len(cluster0_indices)

In [ ]:
cluster0_indices

In [ ]:
# Je crée un vecteur de booléen pour savoir si le modèle i est dans le cluster1 ou non.
cluster1 = npos[:,1] < 2

cluster1_indices = list()
for i in range(0,npos.shape[0]):
    if cluster1[i]:
        cluster1_indices.append(i)
len(cluster1_indices)

In [ ]:
cluster1_indices

In [ ]:
# Je crée un vecteur de booléen pour savoir si le modèle i est dans le cluster2 ou non.
cluster2 = npos[:,0] > 5

cluster2_indices = list()
for i in range(0,npos.shape[0]):
    if cluster2[i]:
        cluster2_indices.append(i)
len(cluster2_indices)

In [ ]:
cluster2_indices

In [ ]:
# Scatterplot coloré par cluster :

categories = list()
for i in range(0,250):
    if cluster0[i]:
        categories.append(0)
    elif cluster1[i]:
        categories.append(1)
    elif cluster2[i]:
        categories.append(2)

correspondance_couleur = {0:'red', 1:'orange', 2:'green'}

f = plt.figure(figsize=(10, 10))
ax = sns.scatterplot(x=npos[:, 0], y=npos[:, 1], hue=categories, palette=correspondance_couleur)
plt.legend(loc='lower right')

**EN COURS**